In [ ]:
from pyspark.sql import functions as f

In [ ]:
# Nessa tarefa, você deverá encontrar os 10 produtos com a maior variação de preço, 
# entre o valor encontrado no varejo online (Americanas) e o estipulado pelo fabricante
df = spark.sql("""
select *
from default.silver
""")

df_2 = df.withColumn('diff', f.when((f.col('retailerPrice') >= f.col('manufacturerPrice')*0.2)\ 
                                    & (f.col('manufacturerPrice') >= f.col('retailerPrice')*0.2)\
                                    , (f.col('retailerPrice')) - (f.col('manufacturerPrice'))))

df_2.groupby(f.col('retailerProductCode'))\
            .agg(f.max(f.col('diff')).alias('max_diff')).sort(f.desc(f.col('max_diff'))).show(10)



In [ ]:
#  e os 10 produtos que apresentam maior indisponibilidade.
display(df.dropna(how='any', subset='titleFlag')\
            .groupby(f.col('retailerProductCode'))\
            .agg(f.sum(
                    f.when((f.col('available')==False) & (f.col('titleFlag')==True), 1)\
                    .when((f.col('available')==True) & (f.col('titleFlag')==True), 0))\
                 .alias('unavailable_count'),
                f.countDistinct(f.col('idRetailerSKU')))\
            .sort(f.desc('unavailable_count')).head(10))